# Introduction to Molecular Dynamics

In classical molecular dynamics (MD), we numerically integrate Newton's equations of motion, to move atoms (or more generally 'particles') through time. 

Recall from physics the relationship $F = ma$, where $F$ is the force, $m$ is the mass and $a$ is acceleration.  Since acceleration is the change in velocity over time, and velocity is the change in position over time, we can therefore evolve the position of particles in the system if we know the forces acting upon them. 

Typically, we think about the forces acting upon particles through consideration of the interaction potential, $U$, which can be related to the force.  The force is defined to be the negative gradient of the interaction potential (i.e., $F = -\nabla U$ ).  

### Interaction Potential

For atomistic systems (i.e., where we explicitly model all the atoms), the interaction potential is typically considered to have two contributions, non-bonded and bonded:

$U_{total} = U_{non-bonded} + U_{bonded}$ 

Where non-bonded interactions consist of: 
$U_{non-bonded} = U_{van der Waals} + U_{charge}$

And bonded interactions describe any topological constraints meant to enforce a given structure of a molecule:

$U_{bonded} = U_{bond} + U_{angle} + U_{dihedral} + U_{improper}$

For the purpose of this introduction, we will focus only modeling a system with $U_{van der waals}$.

### The Lennard-Jones (LJ) Potential

There are many ways the non-bonded van der Waals term can be expressed, although the 12-6 Lennard-Jones potential is the most commonly used:

$U(r) = 4\epsilon\left[ \left(\frac{\sigma}{r}\right)^{12}- \left(\frac{\sigma}{r}\right)^6 \right]$

![LJ potential](img/LJ.png)

In the LJ equation there are two adjustable fitting parameters, $\epsilon$ which dictates the interaction strength, and $\sigma$ which dictates the size of the atom. Note, $r$ is the distance between two particles.


- The short-range $1/r^{12}$ repulsion is used to model the overlap of electron clouds of the atoms. 

- The longer-range $1/r^{6}$ attraction is used to model dispersion interactions.

### Periodic Boundary Conditions
Since, in most cases, MD simulations can only model a very small subset of a real physical system (on the order of nanometers), periodic boundary conditions are typically employed to avoid artifacts associated with hard boundaries.  This results in an infinite, yet periodic system.

As a particle "leaves" a simulation box, it re-enters on the opposite side. Explicit self-interactions are typically not allowed (that is, you would not calculate the potential between a given particle and its periodic image).  

![2d cartoon of periodic images, currently borrowed from this source: http://www.mdpi.com/ijms/ijms-10-05135/article_deploy/html/images/ijms-10-05135f11-1024.png](img/periodic_images.png)

#### Minimum Image Convention
MD simulations typically employ what is known as the "minimum image convention."  For example, consider two particles (A and B) in a cubic simulation box of edge length 10.  Particle A is located at [1,0,0] and Particle B is located at [8,0,0].  The distance between the particles within the simulation box is therefore 7. However, the distance between Particle A and the periodic image of Particle B is actually 3.  Thus, with the minimum image convenction we would consider the distance between these two particles to be 3.  In more algorithmic terms, if the distance between two particles is greater than half the box length, the length of the box is subtracted from the distance between the particles.

![2d cartoon of the minimum image convention, currently borrowed from this source: http://press-files.anu.edu.au/downloads/press/p47571/images/ch06-fig01.jpg](img/min_image.jpg)



## Speeding things up
### Cutoffs
You may notice that the LJ potential shown above, quickly starts approaching zero.  A way to reduce computational cost is to simply ignore any particles that are beyond a certain cutoff distance, typically denoted $r_c$.  The value of this cutoff (and how the potential is actually cutoff), will alter the results slightly.  Surveying the literature, it is common to employ various cutoffs: 

- $2.24\sigma$ (Note: the minimum of the LJ potential occurs at $2^{(1/6)}\sigma \sim 1.12\sigma$, so this is twice that)
- $2.5\sigma$ (Note: this corresponds to a point where the potential is $<$ 1/60 of the strength of the maximum well depth) 
- 12 angstroms, e.g., when dealing with a fully atomistic system. (Note: in atomistic systems, a single cutoff is often used for all species and not modulated by the characteristic $\sigma$ of an atom. For reference, $\sigma$ for carbon is ~3.5 for most models)

It is often not desirable to simply truncate the potential, as this woule create a discontinuity in the force (that could create an "impulse" as particles go from not interacting, to interacting). One approach is to shift the potential such that the value is zero at the cutoff; considering the LJ potential:

$U(r) = 4\epsilon\left[ \left(\frac{\sigma}{r}\right)^{12}- \left(\frac{\sigma}{r}\right)^6 \right] + U(r_c)$

Other approachs, such as smoothing algorithms that do not alter the well-depth, can also be used (e.g., see xplor in HOOMD-Blue).

### Neighborlists
While employing a cutoff will reduce the number of times the code must calculate the interaction potential, calculating the distance between all pairs of particles in the system is still costly.  To avoid this calculate, a neighborlist can be constructed that includes all particles within the cutoff + buffer (the buffer is often called the "skin"). The general idea is that the local environment of a particle changes rather slowly, and thus the same neighborlist can be applied multiple times before needing to be reconstructed.   The frequency of reconstruction will depend on numerous factors:
- have fast is the system changing (often a result of the phase e.g., gases will change much more rapidly than dense fluids or temperature, e.g., high temperature systems will have particles moving faster)
- the size of the buffer or skin (if this is set too small, the neighborlist will need to be rebuilt very frequently, if too large, the computational savings will be minimal)

NOTE, improper parameters given to a neighborlist can result in interactions being missed and systems crashing. Simulation packages often include checks when the list is rebuilt, to identify cases where a particle may have moved "too much" (i.e., greater than the size of the skin) and thus interactions may have been missed. These are often referred to as "Dangerous Builds." 

Often times, the frequency can be set to occur at a set time interval, or be made dynamic (including an additional check that if any particle moves more than half the size of the buffer, the list should be rebuilt).  Various algorithms can be used for speeding up construction of the neighborlist (appropriateness depends on the application). Quoting from the HOOMD-Blue documentation (http://hoomd-blue.readthedocs.io/en/stable/module-md-nlist.html):  

The simplest way to build a neighbor list is O(N^2): each particle loops over all other particles and only includes those within the neighbor list cutoff. This algorithm is no longer implemented in HOOMD-blue because it is slow and inefficient. Instead, three accelerated algorithms based on cell lists and bounding volume hierarchy trees are implemented. The cell list implementation is fastest when the cutoff radius is similar between all pair forces (smaller than 2:1 ratio). The stencil implementation is a different variant of the cell list, and is usually fastest when there is large disparity in the pair cutoff radius and a high number fraction of particles with the bigger cutoff (at least 30%). The tree implementation is faster when there is large size disparity and the number fraction of big objects is low. Because the performance of these algorithms depends sensitively on your system and hardware, you should carefully test which option is fastest for your simulation.
